# Generative AI with Python

## Large Language Models (LLMs)

### Generating Text

### Vision Language Models (VLMs)

### Small Language Models

Language Models come in very small sizes too. Some examples include `smollm` and `tinyllama`. While these models are more prone to hallucination, and have more limited "intelligence," they can run quite fast on less powerful hardware such as Raspberry Pis. This can be useful if you don't have a machine with a GPU, or want something that can create good-enough responses.

## Text to Image with StableDiffusion

## Other ML Tools